### Importando bibliotecas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from collections import Counter
import time

### Dados

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# Base já com as colunas novas adicionadas apartir do mapeamento entre a antiga e a nova
microdados = pd.read_parquet('/content/drive/MyDrive/PIBIC/dadosCenso2018_ComNovasColunas.parquet')

In [ ]:
# Preenchendo valores Nan na coluna de Financiamento Estudantil, com o valor de -1, como no TED
microdados['IN_FINANCIAMENTO_ESTUDANTIL'] = microdados['IN_FINANCIAMENTO_ESTUDANTIL'].fillna(-1)

#### Criando o dataset com as novas colunas

In [ ]:
colunas_necessarias = ['CO_IES', 'CO_CURSO' 'CO_MUNICIPIO', 'CO_UF', 'CO_CINE_AREA_GERAL']
colunas_desejadas = ['ID_ALUNO', 'IN_FINANCIAMENTO_ESTUDANTIL','IN_DEFICIENCIA', 'NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CURSO', 'TP_CATEGORIA_ADMINISTRATIVA']

In [ ]:
microdados_antigos = pd.read_parquet('/content/drive/MyDrive/PIBIC/Dados/dataCenso2018.parquet', columns=colunas_desejadas)
microdados_novos_curso = pd.read_parquet('/content/drive/MyDrive/PIBIC/Dados/microdados_novos_curso.parquet')

In [ ]:
microdados_antigos.drop(['NU_IDADE', 'FAIXA_ETARIA'], axis=1, inplace=True)

In [ ]:
# Mapeamento para adicionar CO_CINE_AREA_GERAL aos dados antigos
map_cine = microdados_novos_curso[['CO_CINE_AREA_GERAL', 'CO_CURSO']].drop_duplicates('CO_CURSO')

In [ ]:
microdados_final = microdados_antigos.merge(map_cine, on='CO_CURSO', how='left')

In [ ]:
# Mapeamento para adicionar CO_MUNICIPIO e CO_UF aos dados antigos
map_ies = microdados_novos_curso[['CO_IES', 'CO_MUNICIPIO', 'CO_UF']].drop_duplicates('CO_IES')

In [ ]:
microdados_final = microdados_final.merge(map_ies, on='CO_IES', how='left')

In [ ]:
microdados_final.to_parquet('/content/drive/MyDrive/PIBIC/dadosCenso2018_ComNovasColunas.parquet')

### Experimento 4 - Substituição do Quase-identificador “CO_CURSO”

#### Código baseado nos conceitos matemáticos (TED)

In [ ]:
def suc_prior_deterministic(data, sensitive_attr):
    """
    Sucesso determinístico a priori do adversário em um ataque ACU.
    """
    sensitive_values = data[sensitive_attr].unique()
    return 1 if len(sensitive_values) == 1 else 0

def suc_post_deterministic(data, qid_attrs, sensitive_attr):
    """
    Sucesso determinístico a posteriori do adversário em um ataque ACU.
    """
    groups = data.groupby(qid_attrs, dropna=False)  # Agrupa pelos QID e inclui NaN como grupo distinto
    total_records = len(data)  # Número total de registros
    success_count = 0  # Contador de sucessos

    for _, group in groups:  # Percorre cada grupo
        if len(group[sensitive_attr].unique()) == 1:  # Se o grupo tem apenas 1 valor sensível
            success_count += len(group)  # Conta todos os registros do grupo como sucesso

    return success_count / total_records  # Retorna a proporção de acertos

def suc_prior_probabilistic(data, sensitive_attr):
    """
    Sucesso probabilístico a priori do adversário em um ataque ACU.
    """
    value_counts = data[sensitive_attr].value_counts()
    total_records = len(data)
    return value_counts.max() / total_records if total_records > 0 else 0

def suc_post_probabilistic(data, qid_attrs, sensitive_attr):
    """
    Sucesso probabilístico a posteriori do adversário em um ataque ACU.
    """
    groups = data.groupby(qid_attrs, dropna=False)  # Inclui NaN como grupo distinto
    total_records = len(data)
    total_success = 0

    for _, group in groups:
        group_size = len(group)
        if group_size > 0:
            value_counts = group[sensitive_attr].value_counts(dropna=False)  # Inclui NaN ao contar
            total_success += value_counts.max()

    return total_success / total_records if total_records > 0 else 0

def degradation_privacy_deterministic(data, qid_attrs, sensitive_attr):
    """
    Degradação determinística de privacidade em um ataque ACU.
    """
    prior = suc_prior_deterministic(data, sensitive_attr)
    post = suc_post_deterministic(data, qid_attrs, sensitive_attr)
    return post - prior

def degradation_privacy_probabilistic(data, qid_attrs, sensitive_attr):
    """
    Degradação probabilística de privacidade em um ataque ACU.
    """
    prior = suc_prior_probabilistic(data, sensitive_attr)
    post = suc_post_probabilistic(data, qid_attrs, sensitive_attr)
    return post / prior if prior > 0 else 0

In [ ]:
quase_identificadores_list = [
    microdados[['CO_CURSO']],
    microdados[['CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']]
]

##### Atributo Sensível [IN_FINANCIAMENTO_ESTUDANTIL]

###### Substituição de CO_CURSO por CO_CINE

In [ ]:
qid_attrs = ['CO_CINE_AREA_GERAL']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['CO_MUNICIPIO_NASCIMENTO', 'CO_CINE_AREA_GERAL']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'CO_CINE_AREA_GERAL']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CINE_AREA_GERAL']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CINE_AREA_GERAL']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CINE_AREA_GERAL']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CINE_AREA_GERAL']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CINE_AREA_GERAL']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

###### Substituição de CO_CURSO por CO_MUNICIPIO

In [ ]:
qid_attrs = ['CO_MUNICIPIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['CO_MUNICIPIO_NASCIMENTO', 'CO_MUNICIPIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'CO_MUNICIPIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_MUNICIPIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_MUNICIPIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_MUNICIPIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

###### Substituição de CO_CURSO por CO_UF

In [ ]:
qid_attrs = ['CO_UF']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['CO_MUNICIPIO_NASCIMENTO', 'CO_UF']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'CO_UF']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_UF']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_UF']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_UF']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_UF']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_UF']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

###### Substituição de CO_CURSO por CO_IES

In [ ]:
qid_attrs = ['CO_IES']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['CO_MUNICIPIO_NASCIMENTO', 'CO_IES']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'CO_IES']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_IES']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_IES']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_IES']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_IES']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_IES']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

###### Substituição de CO_CURSO por TP_CATEGORIA_ADMINISTRATIVA

In [ ]:
qid_attrs = ['TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['CO_MUNICIPIO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO', 'TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

##### Atributo Sensível [IN_DEFICIENCIA]

###### Substituição de CO_CURSO por CO_CINE

In [ ]:
qid_attrs = ['CO_CINE_AREA_GERAL']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['CO_MUNICIPIO_NASCIMENTO', 'CO_CINE_AREA_GERAL']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'CO_CINE_AREA_GERAL']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CINE_AREA_GERAL']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CINE_AREA_GERAL']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CINE_AREA_GERAL']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CINE_AREA_GERAL']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CINE_AREA_GERAL']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

###### Substituição de CO_CURSO por CO_MUNICIPIO

In [ ]:
qid_attrs = ['CO_MUNICIPIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['CO_MUNICIPIO_NASCIMENTO', 'CO_MUNICIPIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'CO_MUNICIPIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_MUNICIPIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_MUNICIPIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_MUNICIPIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

###### Substituição de CO_CURSO por CO_UF

In [ ]:
qid_attrs = ['CO_UF']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['CO_MUNICIPIO_NASCIMENTO', 'CO_UF']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'CO_UF']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_UF']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_UF']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_UF']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_UF']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_UF']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

###### Substituição de CO_CURSO por CO_IES

In [ ]:
qid_attrs = ['CO_IES']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['CO_MUNICIPIO_NASCIMENTO', 'CO_IES']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'CO_IES']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_IES']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_IES']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_IES']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_IES']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_IES']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

###### Substituição de CO_CURSO por TP_CATEGORIA_ADMINISTRATIVA

In [ ]:
qid_attrs = ['TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['CO_MUNICIPIO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO', 'TP_CATEGORIA_ADMINISTRATIVA']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")